In [15]:
#!/usr/bin/env python3
"""
Python script to automatically generate MATLAB scripts for NEVIS regional model
with adjustable parameters: oo.mean_perms and pd.B_reg

Author: Auto-generated script
Date: 2025-09
"""

import os
import argparse
import sys
import numpy as np
from pathlib import Path

def format_scientific(value):
    if value >= 1:
        sci_str = f"{value:.0e}".replace('e+0', 'e').replace('e+', 'e')
    else:
        sci_str = f"{value:.0e}"
    
    return sci_str.replace('-', '_').replace('+', '')

def generate_spinup_script(kappa=1e-10, t_drainage=300, V=0, mean_perms=1, H_reg_value=1000, mu=1e2, output_dir="./", case_suffix="_spinup"):
    """
    Generate MATLAB script with specified parameters
    
    Parameters:
    -----------
    kappa : float
        Value for leakage coeff
    mean_perms : int
        Value for oo.mean_perms (0 or 1)
    H_reg_value : float
        Thickness for pd.B_reg (default: 1000 gives original value)
    output_dir : str
        Directory to save the generated script
    case_suffix : str
        Suffix to add to the case name
    """
    kappa_str = format_scientific(kappa)
    mu_str = format_scientific(mu)
    # Generate case name based on parameters
    casename = f"n2d_region_melt_meanperms{mean_perms}_Hreg{H_reg_value}_kappa{kappa_str}_mu{mu_str}{case_suffix}"

    matlab_script = f"""
        %% Script to run NEVIS regional model 
        % This script is designed to run the NEVIS 1-dimensional model for an idealised ice sheet
        % The surface runoff is set up by a prescribed function
        % The surface and bed profiles are the same as in Hewitt 2013

        % Author: Auto-generated script  
        % Date: 2025-09
        % Parameters: oo.mean_perms={mean_perms}, pd.H_reg_value=pd.Ye*({H_reg_value})^3/(12*(1-0.33)^2)
        format compact

        %% read in the initial condition
        oo.root = './';                                % filename root
        oo.code = '../nevis/src';                      % code directory   
        oo.results = 'results';                        % path to the results folders
        oo.dataset = 'nevis_regional';                 % dataset name
        oo.casename = '{casename}';    % casename
        oo.fn = ['/',oo.casename];                     % filename (same as casename)
        oo.rn = [oo.root,oo.results,oo.fn];            % path to the case results
        oo.dn = [oo.root, 'data/', oo.dataset, '/'];   % path to the data
        addpath(oo.code);                              % add path to code
        mkdir(oo.rn);                                  % create directory for results 

        %% parameters
        [pd,oo] = nevis_defaults([],oo);  

        oo.evaluate_variables = 1;
        oo.input_gaussian = 1;
        oo.relaxation_term = 1;                         % 0 is alpha hb, 1 is alpha deltap hb
        oo.initial_condition = 1;                       % 1 is default condition from 0365.mat, 0 is using steady-state drainage system, winter or summertime
        oo.mean_perms = {mean_perms};
        oo.modified_mean_perms = 0;
        oo.display_residual =0;
        pd.alpha_b = 0;                                 % relaxation rate (s^-1)
        pd.kappa_b = {kappa};                             % relaxation coeff

        % alter default parmaeters 
        pd.mu = {mu};                                    % water viscosity (Pa s)
        pd.c_e_reg2 = 0.00/1e3/9.81;                    % elastic sheet thickness [m/Pa]
        pd.u_b = 100/pd.ty;                             % sliding speed [m/s]
        pd.sigma = 1e-3;                                % englacial void fraction
        pd.h_r = 0.1;                                   % roughness height [m]
        pd.l_r = 10;                                    % roughness length [m]
        pd.l_c = 1000;                                  % sheet width contributing to conduit melting [m] default = 10 m
        pd.k_s = 1e-3;                                  % sheet permeability constant
        pd.tau_b = 60e3;                                % driving stress [Pa]
        pd.melt = pd.G/pd.rho_w/pd.L;                   % geothermal heat derived basal melt [m/s]
        pd.melt = (pd.G+(pd.u_b*pd.tau_b))/pd.rho_w/pd.L;  
                                                        % geothermal heat + frictional heating derived basal melt [m/s]
        pd.meltinterior = 0*((pd.G+((100/pd.ty)*pd.tau_b))/pd.rho_w/pd.L)*1e3; 
                                                        % flux of basal melt up to the ~icedivide (200 km) [m2/s]

        pd.hb_reg1 = 5e-3;                              % Regularisation parameter for hb in blister propagation
        pd.hb_reg2 = 1e-3;                              % Regularisation parameter for hb in blister leakage
        pd.N_reg1 = 1e4;                                % Regularisation parameter for N in blister leakage
        pd.deltap_reg = 1e4;                            % Regularisation parameter for deltap
        pd.B_reg = pd.Ye*({H_reg_value})^3/(12*(1-0.33)^2);      % Reg parameter for bending stiffness

        % non-dimensionalise
        ps = struct;
        [ps,pp] = nevis_nondimension(pd,ps,oo);

        %% grid and geometry
        load([oo.dn '/' 'morlighem_for_nevis_140km']); % load Morlighem bedmap (previously collated)
        dd = morlighem_for_nevis_140km; dd.skip = 6;
        gg = nevis_grid(dd.X_km(1:dd.skip:end,1)/ps.x,dd.Y_km(1,1:dd.skip:end)/ps.x,oo); 
        b = reshape(dd.B_km(1:dd.skip:end,1:dd.skip:end)/ps.z,gg.nIJ,1);
        s = reshape(dd.S_km(1:dd.skip:end,1:dd.skip:end)/ps.z,gg.nIJ,1);
        % save a copy of the bed and surface
        gg.bed = b;
        gg.surface = s;

        x = dd.X_km(1:dd.skip:end,1)/ps.x;
        y = dd.Y_km(1,1:dd.skip:end)/ps.x;
        
        %% mask with minimum ice thickness
        H = max(s-b,0);
        Hmin = 0/ps.z; 
        nout = find(H<=Hmin);
        gg = nevis_mask(gg,nout); 
        gg.n1m = gg.n1;                                   % label all edge nodes as boundary nodes for pressure

        %% label boundary nodes
        gg = nevis_label(gg,gg.n1m);
        oo.adjust_boundaries = 1;                         % enable option of changing conditions

        %% plot grid
        % nevis_plot_grid(gg); return;                    % check to see what grid looks like

        %% initialize variables
        [aa,vv] = nevis_initialize(b,s,gg,pp,oo);         % default initialisation
        pd.k_f = 0.9;                                     % percent overburden (k-factor) 
        vv.phi = aa.phi_a+pd.k_f*(aa.phi_0-aa.phi_a);     % initial pressure  k_f*phi_0
        N = aa.phi_0-vv.phi;                              % N for initial cavity sheet size 
        vv.hs = ((((pd.u_b*pd.h_r/pd.l_r)./((pd.u_b/pd.l_r)+(pd.K_c.*((ps.phi*N).^3)))))./ps.h); % initial cavity sheet size as f(N)

        %% boundary conditions
        aa.phi_b = max(aa.phi_0,aa.phi_a);                % prescribed boundary pressure at overburden or atmospheric

        %% moulins 
        oo.density_moulins = 1;
        oo.keep_all_moulins = 0;
        % load moulin locations
        tmp = load([oo.dn '/' '/nevis_170207a.mat'],'pp');
        pp.ni_m = tmp.pp.ni_m; pp.sum_m = tmp.pp.sum_m; clear tmp;

        %% supraglacial lakes
        pp.x_l = 40*(1e3/ps.x);                                   % x-coord of lakes
        pp.y_l = -20*(1e3/ps.x);                                  % y-coord of lakes
        pp.V_l = [0/(ps.Q0*ps.t)];                                % volume of lakes
        pp.t_drainage = [{t_drainage}*pd.td/ps.t];                % time of lake drainages (assumed to be the middle time of the Gaussian)
        pp.t_duration = [0.025*pd.td/ps.t];                       % duration of lake drainages, 6hr
        [pp.ni_l,pp.sum_l] = nevis_lakes(pp.x_l,pp.y_l,gg,oo);    % calculate lake catchments

        %% surface runoff
        load([oo.dn '/' 'runoff_2009_nevis140.mat']);               % load data for year of interest (previously collated)
        % RACMO distributed input
        oo.surface_runoff = 0;                                          % If set to 1 turns on surface runoff input
                                                                        % 0 is prescribe moulin input with a function
        oo.RACMO_runoff = 1;                                            % If set to 1 turns on RACMO surface runoff data
                                                                        % 0 is a simple sinusoidal function
        oo.distributed_input = 0;                                       % If set to 1 distributes input across the domain
                                                                        % 0 inputs to discrete moulins

        pp.meltE = @(t) (0/1000/pd.td/ps.m)*(1-exp(-t/(30*pd.td/ps.t))); 
        pp.runoff_function = @(t) 0*runoff(((t*ps.t)/pd.td),runoff_2009_nevis140)./ps.m;  % distributed input (m/sec)
        pp.input_function = @(t) 0;

        %% timestep 
        oo.dt = 1/24*pd.td/ps.t; 
        oo.save_timesteps = 1; 
        oo.save_pts_all = 1; 

        % Add GPS station points downstream of the moulin every 5km
        pp.ni_gps = nevis_gps_array([40e3,40e3,20e3,0,-40e3]/ps.x, [-15e3,-5e3,-15e3,-25e3,-30e3]/ps.x, gg, oo); % GPS station points
        oo.pts_ni = [pp.ni_l' pp.ni_m' pp.ni_gps];    
        oo.t_span = [(1:1:365*5)*pd.td/ps.t];
        % oo.t_span = [(1:1:59)*pd.td/ps.t (59.5:0.001:60.5)*pd.td/ps.t (61:1:120)*pd.td/ps.t];

        %% save initial parameters
        save([oo.rn, oo.fn],'pp','pd','ps','gg','aa','vv','oo');
        [tt,vv,info] = nevis_timesteps(oo.t_span,vv,aa,pp,gg,oo);

        %% expand/update variables
        aa = nevis_inputs(vv.t,aa,vv,pp,gg,oo);
        oo.evaluate_variables = 1; 
        [vv2] = nevis_backbone(inf,vv,vv,aa,pp,gg,oo); 
        vv2 = nevis_nodedischarge(vv2,aa,pp,gg,oo); 
        save([oo.rn, oo.fn],'pp','pd','ps','gg','aa','oo','tt');

        %% Simple animate
        % nevis_regional_animation
        """
    
    # Create output directory if it doesn't exist
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # Write MATLAB script to file
    output_file = os.path.join(output_dir, f"{casename}.m")
    with open(output_file, 'w') as f:
        f.write(matlab_script)
    
    print(f"Generated MATLAB script: {output_file}")
    print(f"  oo.mean_perms = {mean_perms}")
    print(f"  pd.H_reg_value = {H_reg_value}")
    print(f" pd.kappa = {kappa}")
    print(f" pd.mu = {mu}")
    
    return output_file

def generate_drainage_script(kappa=1e-10, t_drainage=300, V=1e7, mean_perms=1, H_reg_value=1000, mu=1e2, output_dir="./", case_suffix="_drainage"):
    """
    Generate MATLAB script for drainage phase with specified parameters
    
    Parameters:
    -----------
    kappa : float
        Value for leakage coeff
    t_drainage : float
        Time for drainage (default: 300 seconds)
    V : float
        Volume for drainage (default: 1e7 m^3)
    mean_perms : int
        Value for oo.mean_perms (0 or 1)
    H_reg_value : float
        Thickness for pd.B_reg (default: 1000 gives original value)
    output_dir : str
        Directory to save the generated script
    case_suffix : str
        Suffix to add to the case name
    """
    
    # Generate case name based on parameters
    kappa_str = format_scientific(kappa)
    mu_str = format_scientific(mu)
    log10V = int(round(np.log10(V)))  # Convert volume to log10 scale
    casename = f"n2d_region_melt_meanperms{mean_perms}_Hreg{H_reg_value}_kappa{kappa_str}_mu{mu_str}_td{t_drainage}_V1e{log10V}{case_suffix}"
    spinup_casename = f"n2d_region_melt_meanperms{mean_perms}_Hreg{H_reg_value}_kappa{kappa_str}_mu{mu_str}_spinup"

    matlab_script = f"""
        %% Script to run NEVIS regional model - DRAINAGE PHASE
        % This script loads spinup results and runs drainage simulation
        % The surface runoff is set up by a prescribed function
        % The surface and bed profiles are the same as in Hewitt 2013

        % Author: Auto-generated script  
        % Date: 2025-09

        format compact

        %% read in the initial condition
        casename = '{casename}'; % drainage system filename
        initname = '{spinup_casename}';

        data = load(['./results/' initname '/' initname]);
        pd = data.pd;                                % load parameters from the initial condition
        ps = data.ps;                                % load state variables from the initial condition
        pp = data.pp;                                % load scaled parameters from the initial condition
        aa = data.aa;                                % load state variables from the initial condition
        oo = data.oo;                                % load options from the initial condition

        oo.casename = casename;                      % drainage system filename
        oo.initname = initname;                      % initial condition filename, for spinup

        oo.fn = ['/',oo.casename];                     % filename (same as casename)
        oo.rn = [oo.root,oo.results,oo.fn];            % path to the case results
        oo.dn = [oo.root, 'data/', oo.dataset, '/'];   % path to the data
        addpath(oo.code);                              % add path to code
        mkdir(oo.rn);                                  % create directory for results    
        pp.c0 = 0;

        %% grid and geometry
        load([oo.dn '/' 'morlighem_for_nevis_140km']); % load Morlighem bedmap (previously collated)
        dd = morlighem_for_nevis_140km; dd.skip = 6;
        gg = nevis_grid(dd.X_km(1:dd.skip:end,1)/ps.x,dd.Y_km(1,1:dd.skip:end)/ps.x,oo); 
        b = reshape(dd.B_km(1:dd.skip:end,1:dd.skip:end)/ps.z,gg.nIJ,1);
        s = reshape(dd.S_km(1:dd.skip:end,1:dd.skip:end)/ps.z,gg.nIJ,1);
        % save a copy of the bed and surface
        gg.bed = b;
        gg.surface = s;

        x = dd.X_km(1:dd.skip:end,1)/ps.x;
        y = dd.Y_km(1,1:dd.skip:end)/ps.x;
        
        %% mask with minimum ice thickness
        H = max(s-b,0);
        Hmin = 0/ps.z; 
        nout = find(H<=Hmin);
        gg = nevis_mask(gg,nout); 
        gg.n1m = gg.n1;                                   % label all edge nodes as boundary nodes for pressure

        %% label boundary nodes
        gg = nevis_label(gg,gg.n1m);
        oo.adjust_boundaries = 1;                         % enable option of changing conditions

        %% plot grid
        % nevis_plot_grid(gg); return;                    % check to see what grid looks like

        %% initialize variables
        init_cond = load(['./results/' oo.initname '/' '1825.mat']); % load initial condition
        vv = init_cond.vv;                                % load state variables from the initial condition

        %% supraglacial lakes
        % pp.x_l = [0.5*(x(1)+x(end))];                                 % x-coord of lakes
        % pp.y_l = [0.5*(y(1)+y(end))];                                 % y-coord of lakes
        pp.x_l = 40*(1e3/ps.x);                                         % x-coord of lakes
        pp.y_l = -20*(1e3/ps.x);                                        % y-coord of lakes
        pp.V_l = [{V}/(ps.Q0*ps.t)];                                    % volume of lakes         
        pp.t_drainage = [{t_drainage}*pd.td/ps.t];                      % time of lake drainages (assumed to be the middle time of the Gaussian)
        pp.t_duration = [0.025*pd.td/ps.t];                              % duration of lake drainages, 6hr
        [pp.ni_l,pp.sum_l] = nevis_lakes(pp.x_l,pp.y_l,gg,oo);          % calculate lake catchments  

        %% surface runoff
        load([oo.dn '/' 'runoff_2010_nevis140.mat']);                    % load data for year of interest (previously collated)
        % RACMO distributed input
        oo.surface_runoff = 0;                                           % If set to 1 turns on surface runoff input
                                                                         % 0 is prescribe moulin input with a function
        oo.RACMO_runoff = 1;                                             % If set to 1 turns on RACMO surface runoff data
                                                                         % 0 is a simple sinusoidal function
        oo.distributed_input = 0;                                        % If set to 1 distributes input across the domain
                                                                         % 0 inputs to discrete moulins
        pp.runoff_function = @(t) 0*runoff(((t*ps.t)/pd.td),runoff_2010_nevis140)./ps.m;  % distributed input (m/sec)
        pp.input_function = @(t) 0;

        %% timestep 
        oo.dt = 1/24*pd.td/ps.t; 
        oo.save_timesteps = 1; 
        oo.save_pts_all = 1; 

        % Add GPS station points downstream of the moulin every 5km
        pp.ni_gps = nevis_gps_array([40e3,40e3,20e3,0,-40e3]/ps.x, [-15e3,-5e3,-15e3,-25e3,-30e3]/ps.x, gg, oo); % GPS station points
        oo.pts_ni = [pp.ni_l' pp.ni_m' pp.ni_gps];    
        oo.t_span = [(1:1:29)*pd.td/ps.t (29.9:0.001:30.1)*pd.td/ps.t (31:1:120)*pd.td/ps.t];  

        %% save initial parameters
        save([oo.rn, oo.fn],'pp','pd','ps','gg','aa','vv','oo');
        [tt,vv,info] = nevis_timesteps(oo.t_span,vv,aa,pp,gg,oo);

        %% expand/update variables
        aa = nevis_inputs(vv.t,aa,vv,pp,gg,oo);
        oo.evaluate_variables = 1; 
        [vv2] = nevis_backbone(inf,vv,vv,aa,pp,gg,oo); 
        vv2 = nevis_nodedischarge(vv2,aa,pp,gg,oo); 
        save([oo.rn, oo.fn],'pp','pd','ps','gg','aa','oo','tt');

        %% Simple animate
        % nevis_regional_animation
        """
    
    # Create output directory if it doesn't exist
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # Write MATLAB script to file
    output_file = os.path.join(output_dir, f"{casename}.m")
    with open(output_file, 'w') as f:
        f.write(matlab_script)
    
    print(f"Generated MATLAB drainage script: {output_file}")
    print(f"  t_drainage = {t_drainage} seconds")
    print(f"  V = {V} m^3")
    print(f"  oo.mean_perms = {mean_perms}")
    print(f"  pd.H_reg_value = {H_reg_value}")
    print(f"  pd.kappa = {kappa}")
    print(f"  pd.mu = {mu}")
    print(f"  Loads from spinup: {spinup_casename}")
    
    return output_file

def batch_generate_drainage(param_combinations, output_dir="./generated_scripts/case_studys"):
    """
    Generate multiple MATLAB drainage scripts with different parameter combinations
    
    Parameters:
    -----------
    param_combinations : list of tuples
        List of (kappa, t_drainage, mean_perms, H_reg_value) combinations
    output_dir : str
        Directory to save all generated scripts
    """
    
    generated_files = []
    
    for i, (kappa, t_drainage, V, mean_perms, H_reg_value, mu) in enumerate(param_combinations):
        case_suffix = "_drainage"
        output_file = generate_drainage_script(
            kappa=kappa,
            mu=mu,
            t_drainage=t_drainage,
            V=V,
            mean_perms=mean_perms,
            H_reg_value=H_reg_value,
            output_dir=output_dir,
            case_suffix=case_suffix
        )
        generated_files.append(output_file)
    
    return generated_files

def batch_generate_spinup(param_combinations, output_dir="./generated_scripts/case_studys"):
    """
    Generate multiple MATLAB scripts with different parameter combinations
    
    Parameters:
    -----------
    param_combinations : list of tuples
        List of (mean_perms, H_reg_value) combinations
    output_dir : str
        Directory to save all generated scripts
    """
    
    generated_files = []
    
    for i, (kappa, t_drainage, V, mean_perms, H_reg_value, mu) in enumerate(param_combinations):
        case_suffix = "_spinup"  # or any other suffix as needed
        output_file = generate_spinup_script(
            kappa=kappa,
            t_drainage=t_drainage,
            V=V,
            mu=mu,
            mean_perms=mean_perms,
            H_reg_value=H_reg_value,
            output_dir=output_dir,
            case_suffix=case_suffix
        )
        generated_files.append(output_file)
    
    return generated_files

def generate_paired_scripts(param_combinations, output_dir="./generated_scripts/case_studys"):
    """
    Generate both spinup and drainage scripts for each parameter combination
    
    Parameters:
    -----------
    param_combinations : list of tuples
        List of (kappa, mean_perms, H_reg_value) combinations
    output_dir : str
        Directory to save all generated scripts
    """
    
    # Generate spinup scripts
    print("Generating spinup scripts...")
    spinup_files = batch_generate_spinup(param_combinations, output_dir)

    # Generate corresponding drainage scripts
    print("Generating drainage scripts...")
    drainage_files = batch_generate_drainage(param_combinations, output_dir)
    
    print(f"\nGenerated {len(spinup_files)} paired script sets:")
    for spinup, drainage in zip(spinup_files, drainage_files):
        print(f"  Spinup:   {spinup}")
        print(f"  Drainage: {drainage}")
        print()
    
    return spinup_files, drainage_files

def main():
    # Check if running in Jupyter
    if 'ipykernel' in sys.modules:
        # Running in Jupyter - use default parameters or direct function calls
        print("Running in Jupyter environment")
        return
    
    parser = argparse.ArgumentParser(description='Generate MATLAB scripts for NEVIS regional model')
    parser.add_argument('--kappa', type=float, default=1e-99,
                       help='Value for leakage coeff')
    parser.add_argument('--mu', type=float, default=1e1,
                       help='Value for specific storage (default: 1e1)')
    parser.add_argument('--t_drainage', type=float, default=300,
                       help='Time for drainage (default: 300 seconds)')
    parser.add_argument('--V', type=float, default=1e7,
                       help='Volume for drainage (default: 1e7 m^3)')
    parser.add_argument('--mean_perms', type=int, default=1, choices=[0, 1],
                       help='Value for oo.mean_perms (0 or 1)')
    parser.add_argument('--H_reg_value', type=float, default=1000,
                       help='Thickness for pd.B_reg (default: 1000)')
    parser.add_argument('--output_dir', type=str, default='./',
                       help='Output directory for generated scripts')
    parser.add_argument('--batch', action='store_true',
                       help='Generate batch of scripts with different parameters')
    parser.add_argument('--paired', action='store_true',
                       help='Generate paired spinup and drainage scripts')
    
    args = parser.parse_args()
    
    if args.batch:
        # Define parameter combinations for batch generation
        param_combinations = [
            (1e-10, 30, 5e8, 1, 1000, 1e1),
            (1e-10, 30, 1e9, 1, 1000, 1e2),
            (1e-10, 30, 1e9, 1, 1000, 1e3),
            (1e-10, 300, 1e9, 1, 1000, 1e1),
            (1e-10, 300, 1e9, 1, 1000, 1e2),
            (1e-10, 300, 1e9, 1, 1000, 1e3),
            (1e-99, 30, 1e9, 1, 1000, 1e1),
            (1e-99, 30, 1e9, 1, 1000, 1e2),
            (1e-99, 30, 1e9, 1, 1000, 1e3),
            (1e-9, 30, 1e9, 1, 1000, 1e1),
            (1e-9, 30, 1e9, 1, 1000, 1e2),
            (1e-9, 30, 1e9, 1, 1000, 1e3),
        ]
        
        if args.paired:
            print("Generating paired spinup and drainage scripts...")
            spinup_files, drainage_files = generate_paired_scripts(param_combinations, args.output_dir)
            # print(f"\nGenerated {len(spinup_files)} spinup and {len(drainage_files)} drainage scripts")
        else:
            print("Generating batch of spinup scripts...")
            generated_files = batch_generate_spinup(param_combinations, args.output_dir)
            print(f"\nGenerated {len(generated_files)} MATLAB scripts:")
            for file in generated_files:
                print(f"  {file}")
                
    else:
        # Generate single script  
        if args.paired:
            # Generate paired scripts for single parameter set
            param_combinations = [(args.kappa, args.mean_perms, args.H_reg_value)]
            spinup_files, drainage_files = generate_paired_scripts(param_combinations, args.output_dir)
            print(f"Generated paired scripts:")
            print(f"  Spinup:   {spinup_files[0]}")
            print(f"  Drainage: {drainage_files[0]}")
        else:
            # Generate single spinup script
            output_file = generate_spinup_script(
                kappa=args.kappa,
                mean_perms=args.mean_perms,
                H_reg_value=args.H_reg_value,
                output_dir=args.output_dir
            )
            print(f"Generated single spinup script: {output_file}")

if __name__ == "__main__":
    main()
# 
param_combinations = [
    (1e-99, 60, 1.8e8, 1, 1000, 2e1), 
    (1e-99, 60, 1.8e8, 1, 1000, 1e1), 
    (5e-12, 60, 1.8e8, 1, 1000, 1e1), # mu=1e1
    (1e-11, 60, 1.8e8, 1, 1000, 1e1),
    (5e-11, 60, 1.8e8, 1, 1000, 1e1),
    (1e-10, 60, 1.8e8, 1, 1000, 1e1),
    (5e-13, 60, 1.8e8, 1, 1000, 1e0), # mu=1e0
    (1e-12, 60, 1.8e8, 1, 1000, 1e0),
    (5e-12, 60, 1.8e8, 1, 1000, 1e0),
    (1e-11, 60, 1.8e8, 1, 1000, 1e0),
]

param_combinations = [
    (1e-99, 30, 1.8e8, 1, 1000, 2e1),
    (1e-11, 30, 1.8e8, 1, 1000, 1e1),
    (1e-10, 30, 1.8e8, 1, 1000, 1e1),
    (5e-12, 30, 1.8e8, 1, 1000, 1e1),
    (1e-99, 30, 1.8e8, 1, 1000, 1e1)
]

print("Generating paired spinup and drainage scripts...")
spinup_files, drainage_files = generate_paired_scripts(param_combinations, "./generated_scripts/case_studys/")
print(f"Generated {len(spinup_files)} spinup scripts and {len(drainage_files)} drainage scripts")

Running in Jupyter environment
Generating paired spinup and drainage scripts...
Generating spinup scripts...
Generated MATLAB script: ./generated_scripts/case_studys/n2d_region_melt_meanperms1_Hreg1000_kappa1e_99_mu2e1_spinup.m
  oo.mean_perms = 1
  pd.H_reg_value = 1000
 pd.kappa = 1e-99
 pd.mu = 20.0
Generated MATLAB script: ./generated_scripts/case_studys/n2d_region_melt_meanperms1_Hreg1000_kappa1e_11_mu1e1_spinup.m
  oo.mean_perms = 1
  pd.H_reg_value = 1000
 pd.kappa = 1e-11
 pd.mu = 10.0
Generated MATLAB script: ./generated_scripts/case_studys/n2d_region_melt_meanperms1_Hreg1000_kappa1e_10_mu1e1_spinup.m
  oo.mean_perms = 1
  pd.H_reg_value = 1000
 pd.kappa = 1e-10
 pd.mu = 10.0
Generated MATLAB script: ./generated_scripts/case_studys/n2d_region_melt_meanperms1_Hreg1000_kappa5e_12_mu1e1_spinup.m
  oo.mean_perms = 1
  pd.H_reg_value = 1000
 pd.kappa = 5e-12
 pd.mu = 10.0
Generated MATLAB script: ./generated_scripts/case_studys/n2d_region_melt_meanperms1_Hreg1000_kappa1e_99_mu1e1_s